In [ ]:
from tqdm.notebook import tqdm
from nltk import sent_tokenize
from transformers import AutoTokenizer
import sys
import seaborn as sb
sys.path.append('../..')

from src import *
from src.test_utils import * 

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [ ]:
# gritlm = GritLM("GritLM/GritLM-7B", device_map="cuda:2", torch_dtype="auto")
retriever = Retriever(device='cpu')
# llm = LLM()
# llm = 'mistralai/Mistral-7B-Instruct-v0.2'
llm = None
longdoc = LongDoc(retriever, llm)
# dataset = NarrativeQADataset(llm)
dataset = QualityDataset(llm, split='dev')
# reading_agent = ReadingAgent(dataset, llm)

In [ ]:
test_i = 2
results = [ChunkInfo(**ci) for ci in read_json(os.path.join(dataset.data_dir, f'index_wg_5_2_{test_i}.json'))]
relation_graph = longdoc.build_relation_graph(results)
pages = [ci.passage for ci in results]
questions, answers = dataset.get_questions_and_answers(dataset.data[test_i])
questions = [q.splitlines()[0] for q in questions]
questions

In [ ]:
for ci in results:
    # print(len(gritlm.tokenizer(ci.passage)['input_ids']))
    print(len(retriever.retriever_tokenizer(ci.passage)['input_ids']))

# Index passages

In [ ]:
paragraphs = read_json(os.path.join(dataset.data_dir, f'pages_{2}.json'))

## Eval

In [ ]:
results[11].print()

In [ ]:
results[11].prev_summaries

In [ ]:
print(results[11].recap_str)

# Retrieval

In [ ]:
qid = 5
question = questions[qid]
print(question)

## Eval

### Contriever

#### Query Encode With Note, Doc Encode Without Note

In [ ]:
ent_candidates = longdoc.collect_entities_from_text(questions[qid])
prev_ent_descriptions, prev_relation_descriptions = longdoc.retrieve_descriptions(results, relation_graph, ent_candidates, 1, 2)
q_info = ChunkInfo(len(results), questions[qid], prev_ent_descriptions=prev_ent_descriptions, prev_relation_descriptions=prev_relation_descriptions)
recap_str = f'''Recap:\n{q_info.recap_str}\n\nQuery:\n'''
full_input = recap_str + questions[qid]
print(len(retriever.retriever_tokenizer(full_input)['input_ids']))

q_embedding = retriever.embed_paragraphs([full_input], normalize=False, complete_return=True)
page_embeddings = retriever.embed_paragraphs(pages, normalize=False, complete_return=True)
c_retriever_tokenizer = retriever.retriever_tokenizer
c_q_input_ids, c_q_emb, c_q_lhs = hidden_states_wo_instruction(q_embedding.input_ids.copy(), q_embedding.last_hidden_states.copy(), q_embedding.attention_mask.copy(), c_retriever_tokenizer([recap_str])['attention_mask'], True)
c_p_input_ids, c_p_emb, c_p_lhs = hidden_states_wo_instruction(page_embeddings.input_ids.copy(), page_embeddings.last_hidden_states.copy(), page_embeddings.attention_mask.copy(), c_retriever_tokenizer([''])['attention_mask'], True)
c_pids, c_scores = retriever.dense_retrieval(c_q_emb, c_p_emb, None, normalize=False, return_score=True)
q_spans = word_split(c_q_input_ids[0], c_retriever_tokenizer, False, True)
query_indicatiors(c_retriever_tokenizer, questions[qid], pages, c_q_lhs[0], c_q_input_ids[0], c_p_lhs, c_p_input_ids, c_pids, c_scores, q_spans=q_spans)

#### Query Encode Without Note, Doc Encode Without Note

In [ ]:
recap_str = ''
full_input = recap_str + questions[qid]
print(len(retriever.retriever_tokenizer(full_input)['input_ids']))

q_embedding = retriever.embed_paragraphs([full_input], normalize=False, complete_return=True)
page_embeddings = retriever.embed_paragraphs(pages, normalize=False, complete_return=True)
c_retriever_tokenizer = retriever.retriever_tokenizer
c_q_input_ids, c_q_emb, c_q_lhs = hidden_states_wo_instruction(q_embedding.input_ids.copy(), q_embedding.last_hidden_states.copy(), q_embedding.attention_mask.copy(), c_retriever_tokenizer([recap_str])['attention_mask'], True)
c_p_input_ids, c_p_emb, c_p_lhs = hidden_states_wo_instruction(page_embeddings.input_ids.copy(), page_embeddings.last_hidden_states.copy(), page_embeddings.attention_mask.copy(), c_retriever_tokenizer([''])['attention_mask'], True)
c_pids, c_scores = retriever.dense_retrieval(c_q_emb, c_p_emb, None, normalize=False, return_score=True)
q_spans = word_split(c_q_input_ids[0], c_retriever_tokenizer, False, True)
query_indicatiors(c_retriever_tokenizer, questions[qid], pages, c_q_lhs[0], c_q_input_ids[0], c_p_lhs, c_p_input_ids, c_pids, c_scores)#, q_spans=q_spans)

### GritLM

#### Query Encode With Note, Doc Encode With Note

In [ ]:
ent_candidates = longdoc.collect_entities_from_text(questions[qid])
prev_ent_descriptions, prev_relation_descriptions = longdoc.retrieve_descriptions(results, relation_graph, ent_candidates, 4, True)
q_info = ChunkInfo(len(results), questions[qid], prev_ent_descriptions=prev_ent_descriptions, prev_relation_descriptions=prev_relation_descriptions)
g_q_emb, g_q_input_ids, g_q_lhs = gritlm.encode([questions[qid]], max_length=8192, instructions=[LongDocPrompt.embed_w_note(q_info.recap_str, 'query')])

g_p_emb, g_p_input_ids, g_p_lhs = gritlm.encode(pages, batch_size=5, max_length=8192, instructions=[LongDocPrompt.embed_w_note(ci.recap_str, 'passage') for ci in results])
g_retriever_tokenizer = gritlm.tokenizer
q_spans = word_split(g_q_input_ids[0], g_retriever_tokenizer)
g_pids, g_scores = retriever.dense_retrieval(g_q_emb, g_p_emb, None, normalize=False, return_score=True)
query_indicatiors(g_retriever_tokenizer, questions[qid], pages, g_q_lhs[0], g_q_input_ids[0], g_p_lhs, g_p_input_ids, g_pids, g_scores)#, q_spans=q_spans)

#### Query Encode Without Note, Doc Encode With Note

In [ ]:
g_q_emb, g_q_input_ids, g_q_lhs = gritlm.encode([questions[qid]])#, instructions=["Retrieve relevant passages from a story to answer a given question."])

g_p_emb, g_p_input_ids, g_p_lhs = gritlm.encode(pages, batch_size=5, max_length=8192, instructions=[LongDocPrompt.embed_w_note(ci.recap_str, 'passage') for ci in results])
g_retriever_tokenizer = gritlm.tokenizer
q_spans = word_split(g_q_input_ids[0], g_retriever_tokenizer)
g_pids, g_scores = retriever.dense_retrieval(g_q_emb, g_p_emb, None, normalize=False, return_score=True)
query_indicatiors(g_retriever_tokenizer, questions[qid], pages, g_q_lhs[0], g_q_input_ids[0], g_p_lhs, g_p_input_ids, g_pids, g_scores)#, q_spans=q_spans)

#### Query Encode With Note, Doc Encode Without Note

In [ ]:
ent_candidates = longdoc.collect_entities_from_text(questions[qid])
prev_ent_descriptions, prev_relation_descriptions = longdoc.retrieve_descriptions(results, relation_graph, ent_candidates, 1, 2)

In [ ]:
prev_ent_descriptions

In [ ]:
prev_relation_descriptions

In [ ]:
ent_candidates = longdoc.collect_entities_from_text(questions[qid])
prev_ent_descriptions, prev_relation_descriptions = longdoc.retrieve_descriptions(results, relation_graph, ent_candidates, 1, 2)
q_info = ChunkInfo(len(results), questions[qid], prev_ent_descriptions=prev_ent_descriptions, prev_relation_descriptions=prev_relation_descriptions)
instruction = gritlm.gritlm_instruction('Use the recap context to help you understand the query and retrieve relevant passages from a story to answer the query.')
recap_str = f'''{instruction}\nRecap:\n{q_info.recap_str}\n\nQuery:\n'''
print(len(gritlm.tokenizer(recap_str + questions[qid])['input_ids']))
g_q_emb, g_q_input_ids, g_q_lhs = gritlm.encode([questions[qid]], max_length=8192, instructions=[recap_str])

g_p_emb, g_p_input_ids, g_p_lhs = gritlm.encode(pages, max_length=8192)
g_retriever_tokenizer = gritlm.tokenizer
q_spans = word_split(g_q_input_ids[0], g_retriever_tokenizer)
g_pids, g_scores = retriever.dense_retrieval(g_q_emb, g_p_emb, None, normalize=False, return_score=True)
query_indicatiors(g_retriever_tokenizer, questions[qid], pages, g_q_lhs[0], g_q_input_ids[0], g_p_lhs, g_p_input_ids, g_pids, g_scores, q_spans=q_spans)

In [ ]:
important_page_tokens(g_retriever_tokenizer, questions[qid], pages, g_q_lhs[0], g_q_input_ids[0], g_q_emb[0], g_p_lhs, g_p_input_ids, g_pids, g_scores)

#### Query Encode Without Note, Doc Encode Without Note

In [ ]:
g_q_emb, g_q_input_ids, g_q_lhs = gritlm.encode([questions[qid]], instructions=[gritlm.gritlm_instruction("Retrieve relevant passages from a story to answer a given question.")])

g_p_emb, g_p_input_ids, g_p_lhs = gritlm.encode(pages, max_length=8192)
g_retriever_tokenizer = gritlm.tokenizer
q_spans = word_split(g_q_input_ids[0], g_retriever_tokenizer)
g_pids, g_scores = retriever.dense_retrieval(g_q_emb, g_p_emb, None, normalize=False, return_score=True)
query_indicatiors(g_retriever_tokenizer, questions[qid], pages, g_q_lhs[0], g_q_input_ids[0], g_p_lhs, g_p_input_ids, g_pids, g_scores, q_spans=q_spans)

## Test Code

In [ ]:
doc_split = DocSplit('intfloat/multilingual-e5-large')

In [ ]:
pages = doc_split.split_paragraphs(dataset.get_article(dataset.data[2]), 512 // 3)

In [ ]:
q_emb = retriever.embed_paragraphs([f"query: {q}" for q in ["The Skipper", "The Skipper didn't", "'I won't listen to your advice.' said the Old Man.", "The Skipper didn't follow the advice.", "The Skipper didn't follow the advice. The Skipper is also called 'Old Man' in the passage.", "The Skipper is also called 'Old Man' in the passage. The Skipper didn't follow the advice."]], normalize=True, complete_return=True)
q_strs, q_lhs = [], []
for qid in range(len(q_emb.embeddings)):
    word_spans = word_split(q_emb.input_ids[qid], retriever.retriever_tokenizer, retriever.retriever_tokenizer.bos_token, retriever.retriever_tokenizer.eos_token)
    temp_q_strs, temp_q_lhs = merge_words_and_embeddings(retriever.retriever_tokenizer, q_emb.input_ids[qid], q_emb.last_hidden_states[qid], word_spans, False)
    q_strs.append(temp_q_strs)
    q_lhs.append(temp_q_lhs)

In [ ]:
sents = []
for ci in results:
    for k in ci.ent_descriptions:
        if 'old man' in k.lower():
            sents.append(f'{k}: {ci.ent_descriptions[k]}')

In [ ]:
sents

In [ ]:
xid, yid = 4, 2
fig, ax = plt.subplots(figsize=(10,10))
sb.heatmap((q_lhs[yid] / np.expand_dims(np.linalg.norm(q_lhs[yid], axis=1), axis=1)) @ (q_lhs[xid] / np.expand_dims(np.linalg.norm(q_lhs[xid], axis=1), axis=1)).T, xticklabels=q_strs[xid], yticklabels=q_strs[yid], annot=True, ax=ax)

In [ ]:
xid, yid = 5, 2
fig, ax = plt.subplots(figsize=(10,10))
sb.heatmap((q_lhs[yid] / np.expand_dims(np.linalg.norm(q_lhs[yid], axis=1), axis=1)) @ (q_lhs[xid] / np.expand_dims(np.linalg.norm(q_lhs[xid], axis=1), axis=1)).T, xticklabels=q_strs[xid], yticklabels=q_strs[yid], annot=True, ax=ax)

In [ ]:
align_check:bool
for pid in range(len(pages)-2):
    sub_pages = pages[pid:pid+3]
    ori_sum = sum(len(doc_split.llm_tokenizer(sub_page)['input_ids'])-2 for sub_page in sub_pages)
    cat_sum = len(doc_split.llm_tokenizer(' '.join(sub_pages))['input_ids'])-2
    align_check = ori_sum == cat_sum
    if not align_check:
        break
if align_check:
    p_input_ids = [None] * len(pages)
    reformed_pages = ['passage: ' + ' '.join(pages[pid:pid+3]) for pid in range(len(pages)-2)]
    p_emb = retriever.embed_paragraphs(reformed_pages, complete_return=True)
    pid2embs = defaultdict(list)
    for pid in range(len(pages)-2):
        sub_input_ids = [doc_split.llm_tokenizer(sub_page)['input_ids'][1:-1] for sub_page in pages[pid:pid+3]]
        sub_lens = [len(input_ids) for input_ids in sub_input_ids]
        batched_emb = p_emb.last_hidden_states[pid][3:]
        start_tid = 0
        for sid, sub_len in enumerate(sub_lens):
            pid2embs[pid + sid].append(batched_emb[start_tid:start_tid+sub_len])
            p_input_ids[pid + sid] = sub_input_ids[sid]
            start_tid += sub_len

In [ ]:
len(pid2embs[33])

In [ ]:
p_lhs = [np.array(pid2embs[pid]).mean(0) for pid in range(len(pid2embs))]
p_embeddings = np.array([lhs.mean(0) for lhs in p_lhs])
p_norm = np.linalg.norm(p_embeddings, axis=1)
p_embeddings = p_embeddings / np.expand_dims(p_norm, 1)
p_lhs = [lhs / n for lhs, n in zip(p_lhs, p_norm)]
pids, scores = retriever.dense_retrieval(q_emb.embeddings, p_embeddings, None, normalize=False, return_score=True)
pids

In [ ]:
query_distribution(retriever.retriever_tokenizer, q_emb.last_hidden_states[0], q_emb.input_ids[0], p_lhs, 5, q_spans=word_spans[3:-1])

In [ ]:
pages[13]

In [ ]:
query_indicatiors(retriever.retriever_tokenizer, question, [f'passage: {p}' for p in pages], q_emb.last_hidden_states[0], q_emb.input_ids[0], p_lhs, p_input_ids, pids[:10], scores, 5, q_spans=word_spans)

In [ ]:
p_emb = retriever.embed_paragraphs([f'passage: {p}' for p in pages], normalize=True, complete_return=True)
pids, scores = retriever.dense_retrieval(q_emb.embeddings, p_emb.embeddings, None, normalize=False, return_score=True)
pids

In [ ]:
query_indicatiors(retriever.retriever_tokenizer, question, [f'passage: {p}' for p in pages], q_emb.last_hidden_states[0], q_emb.input_ids[0], p_emb.last_hidden_states, p_emb.input_ids, pids, scores, q_spans=word_spans)

In [ ]:
%matplotlib widget

import matplotlib.pyplot as plt
import numpy as np

from matplotlib.widgets import Cursor

# Fixing random state for reproducibility
np.random.seed(19680801)

fig, ax = plt.subplots(figsize=(8, 6))

x, y = 4*(np.random.rand(2, 100) - .5)
ax.plot(x, y, 'o')
ax.set_xlim(-2, 2)
ax.set_ylim(-2, 2)

# Set useblit=True on most backends for enhanced performance.
cursor = Cursor(ax, useblit=True, color='red', linewidth=2)

plt.show()